In [1]:
# Commented out IPython magic to ensure Python compatibility.
from google.colab import drive,userdata
import os

drive.mount('/content/drive', force_remount=True)
notebook_drive_path = "/content/drive/MyDrive/PyNucleus Project/Capstone Project V2.ipynb"
notebook_filename = "Capstone Project V2.ipynb"

# GitHub details
GITHUB_USERNAME = userdata.get('GITHUB_USERNAME')
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
REPOSITORY_NAME = 'PyNucleus-Model'
REPO_PATH = f"/content/{REPOSITORY_NAME}" # The path where the repo will be cloned

# --- STEP 2: CLONE REPOSITORY (ONLY IF IT DOESN'T EXIST) ---
if not os.path.exists(REPO_PATH):
    print(f"Cloning {REPOSITORY_NAME} repository...")
    repository_url = f"https://{GITHUB_TOKEN}@github.com/{GITHUB_USERNAME}/{REPOSITORY_NAME}.git"
    !git clone {repository_url} {REPO_PATH}
else:
    print("Repository already exists. Skipping clone.")

# --- STEP 3: NAVIGATE INTO REPOSITORY & COPY YOUR NOTEBOOK ---

# The '%cd' command changes the current directory in Colab
%cd {REPO_PATH}

# --- CONFIGURE GIT USER HERE ---
print("Configuring Git user...")
!git config user.name "{GITHUB_USERNAME}"
!git config user.email "{GITHUB_USERNAME}@users.noreply.github.com"

# print("Setup complete. You are ready to work.")

KeyboardInterrupt: 

In [ ]:
# --- Installation Cell (Run this ONCE per session) ---
print("Installing all required packages...")

# Core LangChain and Community Packages
!pip install -q langchain langchain-core langchain-community langchain-text-splitters

# Document Loading & Processing (Unstructured handles many file types including OCR)
!pip install -q "unstructured[local-inference]"

# LLM & ML Libraries
!pip install -q transformers accelerate bitsandbytes torch sentence-transformers

# Vector Stores
!pip install -q chromadb faiss-cpu # faiss-gpu if you have a Pro Colab with a good GPU

# Data Handling & Utilities
!pip install -q pandas numpy tqdm PyYAML

print("All packages installed successfully.")

In [ ]:
# --- Core Utilities & File Handling ---
import getpass
import yaml # Make sure PyYAML is installed
from pathlib import Path

# --- Data Handling & Progress Bars ---
import pandas as pd
import numpy as np
from tqdm import tqdm

# --- Document Loading ---
# UnstructuredFileLoader is now in langchain_community
from langchain_community.document_loaders import UnstructuredFileLoader
# For future testing of more specific loaders:
# from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader

# --- OCR Libraries ---
# No direct imports needed here if using UnstructuredFileLoader with local-inference,
# as it handles OCR internally.
# You would only import these for a manual OCR process:
# import pytesseract
# from pdf2image import convert_from_path
# from PIL import Image

# --- Core ML & LLM Libraries ---
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes
import accelerate # Often used with transformers

# --- Vector Stores & Embeddings ---
# Using ChromaDB as the primary vector store.
import chromadb
from langchain_community.vectorstores import Chroma # Chroma is now in langchain_community
from langchain_community.embeddings import HuggingFaceEmbeddings # Embeddings are also in langchain_community
# For future testing of a high-performance alternative:
# import faiss
# from langchain_community.vectorstores import FAISS

# --- RAG/Agent Frameworks ---
# Core LangChain components
import langchain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# For future testing of an alternative RAG-focused toolkit:
# from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

# **Data Ingestion and Preprocessing for RAG**

# This is the last cell of the code

In [ ]:
# # --- STEP 3: Add, Commit, and Push Your Changes ---

# # For this example, let's create a new file to commit
# with open("update_log.txt", "a") as f:
#     from datetime import datetime
#     f.write(f"Notebook run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

# print("Staging changes...")
# # Add all modified and new files to the staging area
# !git add .

# print("Committing changes...")
# # Commit the staged changes with a descriptive message
# !git commit -m "added new libraries and Set up Python Env"

# print("Pushing changes to GitHub...")
# # Push your committed changes to the 'main' or 'master' branch on GitHub
# !git push

# print("GitHub sync complete!")

print("Copying your notebook into the repository...")
!cp "{notebook_drive_path}" "{notebook_filename_in_repo}"

# Add the notebook file to Git
!git add "{notebook_filename_in_repo}"

# Commit the changes with a message
!git commit -m "Update project notebook with new analysis"

# Push the changes to GitHub
!git push origin main

print("SUCCESS! Your notebook has been updated on GitHub.")